In [17]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, SpatialDropout1D,GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [18]:
import numpy as np
from numpy import asarray
import pandas as pd
# BeautifulSoup is used to remove html tags from the text
from bs4 import BeautifulSoup 
import re # For regular expressions
from nltk.corpus import stopwords
import nltk
from sklearn.preprocessing import OneHotEncoder
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [19]:
def load_lines(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.readlines()
	# close the file
	file.close()
	text = [x.strip() for x in text] 
	while '' in text:
	  text.remove('')
	return text

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [20]:
x_train= load_lines("/home2/satyamishra/DLNLP/convlstm/OneDrive_3_04-11-2020/SST-2.x.train.txt")
y_train = load_lines("/home2/satyamishra/DLNLP/convlstm/OneDrive_3_04-11-2020/SST-2.y.train.txt")
x_dev= load_lines("/home2/satyamishra/DLNLP/convlstm/OneDrive_3_04-11-2020/SST-2.x.dev.txt")
y_dev = load_lines("/home2/satyamishra/DLNLP/convlstm/OneDrive_3_04-11-2020/SST-2.y.dev.txt")
len(x_train)

6911

In [ ]:
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename,'r')
	lines = file.readlines()[1:]
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = asarray(parts[1:], dtype='float32')
	return embedding
#raw_embedding is a dictionary
embeddings_index = load_embedding('/home2/satyamishra/DLNLP/glove.6B/glove.6B.200d.txt')

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [ ]:
length = []
for x in x_train:
    length.append(len(x.split()))
max(length)

52

In [ ]:
x_train_seq = pad_sequences(sequences, maxlen=60)


In [ ]:
sequences_val = tokenizer.texts_to_sequences(x_dev)
x_val_seq = pad_sequences(sequences_val, maxlen=60)

In [ ]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train=le.transform(y_train).ravel()
y_dev=le.transform(y_dev).ravel()


In [ ]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model
tweet_input = Input(shape=(1,60,), dtype='int32')

model = Sequential()
model.add(Embedding(100000, 200, weights=[embedding_matrix], input_length=60, trainable=True))
model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 60, 200)           20000000  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 60, 100)           60100     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 30, 100)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 20,221,101
Trainable params: 20,221,101
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:


model.fit(x_train_seq, y_train, batch_size=32, epochs=10,
                     validation_data=(x_val_seq, y_dev))

Train on 6911 samples, validate on 872 samples
Epoch 1/10
6911/6911 [==============================] - 290s 42ms/step - loss: 0.5147 - accuracy: 0.7322 - val_loss: 0.4235 - val_accuracy: 0.8131
Epoch 2/10
6880/6911 [============================>.] - ETA: 1s - loss: 0.2862 - accuracy: 0.8792